# OCR TT DG2 (GPT-4o)

* Zero-Shot, Chain-Of-Thought
* Best Ergebnisse (text_prompts, img_prompts): T1.2, T2.3, T3.3, T4.3

### Setup

In [8]:
from utils.openai import OpenAI
from utils.parser import Parser
from utils.runner import Runner
from utils.pprint import pprint
import asyncio

instructions = "Du bist ein Assistent zur getreuen Wiedergabe von Informationen aus einem Bebauungsplan. Achte auf Vollständigkeit."
ava = OpenAI(instructions)
parser = Parser()
runner = Runner()

In [9]:
pdf_path = "../data/raw/bpläne/2_zeichnung_textteil_getrennt/F11-TT.pdf"
pdf_prompts = parser.pdf2prompts(pdf_path)
print("Seitenzahl:", len(pdf_prompts))

Seitenzahl: 10


### T1) OCR komplettes PDF

In [3]:
instruction = 'Extrahiere den kompletten Textinhalt. Output im LaTeX-Format.'
instruction_prompt = parser.text2prompts([instruction])

In [4]:
# T1.1) OCR via GPT-4o / Kompletter schriftlicher Teil
# Idee: Komplettes PDF auf einmal einlesen. Ziel: Konsistentes Layout.
# ÜBERSCHREITUNG OUTPUT LENGHT: 4803 Tokens, 12620 Zeichen
pdf_text = ava.request([*pdf_prompts, *instruction_prompt])
pprint(pdf_text)

```latex
\documentclass{article}
\usepackage[utf8]{inputenc}
\usepackage{graphicx}

\begin{document}

\begin{center}
\includegraphics[width=0.2\textwidth]{logo.png} \\
\textbf{STADT LAICHINGEN} \\
\textbf{ALB-DONAU-KREIS} \\
\end{center}

\begin{flushright}
19.06.2000 \\
9833 \\
\end{flushright}

\section*{SCHRIFTLICHER TEIL (Teil B)}
\textbf{BEBauungsplan "GEWERBEGEBIET HIMMELREICH"} \\
\textbf{STADT LAICHINGEN, GEMARKUNG FELDSTETTEN, ALB-DONAU-KREIS}

Der Geltungsbereich wird durch das Planzeichen im Lageplan begrenzt. \\
Lageplan M 1: 500

Für die planungsrechtlichen bzw. bauordnungsrechtlichen Festsetzungen gelten:
\begin{itemize}
    \item Baugesetzbuch (BauGB) in der Fassung der Bekanntmachung vom 27.08.1997 (BGBl. I. S. 2141).
    \item Baunutzungsverordnung (BauNVO) in der Fassung der Bekanntmachung vom 23.01.1990 (BGBl. S. 132), zuletzt geändert am 22.04.1993 (BGBl. I. S. 466).
    \item Planzeichenverordnung 1990 (PlanzV 90) in der Fassung der Bekanntmachung vom 18.12.1990 (B

In [6]:
# T1.2) OCR via GPT-4o / Kompletter schriftlicher Teil
# Idee: Jede Seite einzeln verarbeiten. Ziel: Fokus maximieren und Context-Window klein halten + Seitenzahlen beleiben erhalten.
prompt_chain = list(map(lambda prompt: ava.lambdaRequest([prompt, *instruction_prompt]), pdf_prompts))
msgf11pdf = await asyncio.gather(*prompt_chain)
pprint(msgf11pdf)
%store msgf11pdf

```latex
\documentclass{article}
\usepackage[utf8]{inputenc}
\usepackage{graphicx}

\begin{document}

\begin{center}
\includegraphics[width=0.2\textwidth]{logo.png} \\
\textbf{STADT LAICHINGEN} \\
\textbf{ALB-DONAU-KREIS} \\
\end{center}

\begin{flushright}
19.06.2000 \\
9833 \\
\end{flushright}

\begin{center}
\textbf{SCHRIFTLICHER TEIL (Teil B)} \\
\textbf{BEBAUUNGSPLAN "GEWERBEGEBIET HIMMELREICH"} \\
\textbf{STADT LAICHINGEN, GEMARKUNG FELDSTETTEN, ALB-DONAU-KREIS} \\
\end{center}

Der Geltungsbereich wird durch das Planzeichen im Lageplan begrenzt.

Lageplan M 1: 500

Für die planungsrechtlichen bzw. bauordnungsrechtlichen Festsetzungen gelten:

\begin{itemize}
    \item \textbf{Baugesetzbuch (BauGB)} \\
    in der Fassung der Bekanntmachung vom 27.08.1997 (BGBl. I. S. 2141).
    \item \textbf{Baunutzungsverordnung (BauNVO)} \\
    in der Fassung der Bekanntmachung vom 23.01.1990 (BGBl. S. 132), zuletzt geändert am 22.04.1993 (BGBl. I. S. 466).
    \item \textbf{Planzeichenverordnu

### T2) Art der baulichen Nutzung

1. text_prompts
2. img_prompts

In [10]:
%store -r msgbaunvo msgf11pdf
context = parser.text2prompts([msgbaunvo])

In [11]:
# T2.1) Textinformationen: Art der baulichen Nutzung – WITH CONTEXT – TEXT_PROMPTS
# Vector Store
instruction = 'Extrahiere alle planungsrechtlichen Festsetzungen zum Thema "Art der baulichen Nutzung". '

def run():    
    return ava.similaritySearchWithContext(instruction, msgf11pdf, context)
runner.consistency_check(run)

['Hier sind die extrahierten planungsrechtlichen Festsetzungen zum Thema "Art der baulichen Nutzung" aus dem Bebauungsplan "Gewerbegebiet Himmelreich":

### Art der baulichen Nutzung (§§ 1 - 15 BauNVO)

#### Eingeschränktes Gewerbegebiet (GEE) (§ 8 BauNVO i.V.m. § 1 (4) BauNVO)

1. **Zulässig sind / zulässig ist:**
    - **Gewerbebetriebe, die das Wohnen nicht wesentlich stören,** wobei bei Einzelhandelsbetrieben (gem. § 1 (5) BauNVO i.V.m. § 1 (9) BauNVO) nur die nicht innenstadtrelevanten Sortimente Kfz, Motorräder, Mopeds, Fahrräder, Kfz-Zubehör, Rasenmäher, Landmaschinen, Fahrrad- und Motorradzubehör, Brennstoffe und Mineralölerzeugnisse zulässig sind.
    - Der Verkauf von auf dem Grundstück produzierten Waren auf einer untergeordneten Fläche ist zulässig.

2. **Nicht zulässig sind / nicht zulässig ist:**
    - Nutzungen gemäß § 8 (3) Nr. 3 BauNVO (Vergnügungsstätten) sind gemäß § 1 (6) BauNVO nicht Bestandteil des Bebauungsplans und damit nicht zulässig.
    - Einzelhandelsbetrie

[['Hier sind die extrahierten planungsrechtlichen Festsetzungen zum Thema "Art der baulichen Nutzung" aus dem Bebauungsplan "Gewerbegebiet Himmelreich":\n\n### Art der baulichen Nutzung (§§ 1 - 15 BauNVO)\n\n#### Eingeschränktes Gewerbegebiet (GEE) (§ 8 BauNVO i.V.m. § 1 (4) BauNVO)\n\n1. **Zulässig sind / zulässig ist:**\n    - **Gewerbebetriebe, die das Wohnen nicht wesentlich stören,** wobei bei Einzelhandelsbetrieben (gem. § 1 (5) BauNVO i.V.m. § 1 (9) BauNVO) nur die nicht innenstadtrelevanten Sortimente Kfz, Motorräder, Mopeds, Fahrräder, Kfz-Zubehör, Rasenmäher, Landmaschinen, Fahrrad- und Motorradzubehör, Brennstoffe und Mineralölerzeugnisse zulässig sind.\n    - Der Verkauf von auf dem Grundstück produzierten Waren auf einer untergeordneten Fläche ist zulässig.\n\n2. **Nicht zulässig sind / nicht zulässig ist:**\n    - Nutzungen gemäß § 8 (3) Nr. 3 BauNVO (Vergnügungsstätten) sind gemäß § 1 (6) BauNVO nicht Bestandteil des Bebauungsplans und damit nicht zulässig.\n    - Einzel

In [12]:
# T2.3) Textinformationen: Art der baulichen Nutzung – WITH CONTEXT – IMG_PROMPTS
# Komplettes PDF
instruction = 'Extrahiere alle planungsrechtlichen Festsetzungen zum Thema "Art der baulichen Nutzung". Liste am Ende die jeweiligen Seitenzahlen als Referenz auf. Output-Format: ###<Thema>: <Informationen> ### Referenzen: <Seitenzahlen>). '
%store -r msgbaunvo
context = parser.text2prompts([msgbaunvo])

async def run():
    return await ava.extractTextFromImagesWithContexts(instruction, [pdf_path], [context], img_type="pdf")
await runner.async_consistency_check(run)

['### Art der baulichen Nutzung: 
1.1 Eingeschränktes Gewerbegebiet (GEE) (§ 8 BauNVO i.V.m. § 1 (4) BauNVO)
1.1.1 Zulässig sind / zulässig ist:
1.1.1.1 Gewerbebetriebe, die das Wohnen nicht wesentlich stören, wobei bei Einzelhandelsbetrieben (gem. § 1 (5) BauNVO i.V.m. § 1 (9) BauNVO) nur die nicht innenstadtrelevanten Sortimente Kfz, Motorräder, Mopeds, Fahrräder, Kfz-Zubehör, Rasenmäher, Landmaschinen, Fahrrad- und Motorradzubehör, Brennstoffe und Mineralölerzeugnisse zulässig sind.
1.1.1.2 Der Verkauf von auf dem Grundstück produzierten Waren auf einer untergeordneten Fläche ist zulässig.
1.1.2 Nicht zulässig sind / nicht zulässig ist:
1.1.2.1 Nutzungen gemäß § 8 (3) Nr. 3 BauNVO (Vergnügungsstätten) sind gemäß § 1 (6) BauNVO nicht Bestandteil des Bebauungsplans und damit nicht zulässig.
1.1.2.2 Einzelhandelsbetriebe mit innenstadtrelevanten und bestimmten nicht innenstadtrelevanten Sortimenten sind gemäß § 1 (5) BauNVO i.V.m. § 1 (9) BauNVO nicht zulässig.

1.2 Gewerbegebiet (GE) 

[['### Art der baulichen Nutzung: \n1.1 Eingeschränktes Gewerbegebiet (GEE) (§ 8 BauNVO i.V.m. § 1 (4) BauNVO)\n1.1.1 Zulässig sind / zulässig ist:\n1.1.1.1 Gewerbebetriebe, die das Wohnen nicht wesentlich stören, wobei bei Einzelhandelsbetrieben (gem. § 1 (5) BauNVO i.V.m. § 1 (9) BauNVO) nur die nicht innenstadtrelevanten Sortimente Kfz, Motorräder, Mopeds, Fahrräder, Kfz-Zubehör, Rasenmäher, Landmaschinen, Fahrrad- und Motorradzubehör, Brennstoffe und Mineralölerzeugnisse zulässig sind.\n1.1.1.2 Der Verkauf von auf dem Grundstück produzierten Waren auf einer untergeordneten Fläche ist zulässig.\n1.1.2 Nicht zulässig sind / nicht zulässig ist:\n1.1.2.1 Nutzungen gemäß § 8 (3) Nr. 3 BauNVO (Vergnügungsstätten) sind gemäß § 1 (6) BauNVO nicht Bestandteil des Bebauungsplans und damit nicht zulässig.\n1.1.2.2 Einzelhandelsbetriebe mit innenstadtrelevanten und bestimmten nicht innenstadtrelevanten Sortimenten sind gemäß § 1 (5) BauNVO i.V.m. § 1 (9) BauNVO nicht zulässig.\n\n1.2 Gewerbege

### T3) Maß der baulichen Nutzung

1. text_prompts
2. img_prompts

In [20]:
%store -r msgbaunvo msgf11pdf
context = parser.text2prompts([msgbaunvo])

In [21]:
# T3.1) Textinformationen: Maß der baulichen Nutzung – WITH CONTEXT – TEXT_PROMPTS
# Vector Store
instruction = 'Extrahiere alle planungsrechtlichen Festsetzungen zum Thema "Maß der baulichen Nutzung".'

def run():
    return ava.similaritySearchWithContext(instruction, msgf11pdf, context)
runner.consistency_check(run)

['Hier sind die planungsrechtlichen Festsetzungen zum Thema "Maß der baulichen Nutzung" aus dem Bebauungsplan "Gewerbegebiet Himmelreich":

```latex
\\documentclass{article}
\\usepackage[utf8]{inputenc}
\\usepackage{geometry}
\\geometry{a4paper, margin=1in}

\\begin{document}

\\section*{Schriftlicher Teil Bebauungsplan "Gewerbegebiet Himmelreich"\\\\
Stadt Laichingen, Gemarkung Feldstetten, Alb-Donau-Kreis, 19.06.2000}

\\subsection*{1.2 Maß der baulichen Nutzung (§ 16 - 21 a BauNVO)}

\\subsubsection*{1.2.1 Grundflächenzahl (§ 19 BauNVO)}
- siehe Einträge im Lageplan -

\\subsubsection*{1.2.2 Geschossflächenzahl (§ 20 BauNVO)}
- siehe Einträge im Lageplan -

\\subsubsection*{1.2.3 Höhe der baulichen Anlagen (§ 18 BauNVO)}
- siehe Einträge im Lageplan -
Die Gebäudehöhe wird gemessen von der Erdgeschossfußbodenhöhe (EFH) bis zur Schnittkante zwischen Außenwand und Dachhaut bzw. bis zur Oberkante Firstziegel.

\\end{document}
```

Diese Festsetzungen umfassen die Grundflächenzahl, die G

[['Hier sind die planungsrechtlichen Festsetzungen zum Thema "Maß der baulichen Nutzung" aus dem Bebauungsplan "Gewerbegebiet Himmelreich":\n\n```latex\n\\documentclass{article}\n\\usepackage[utf8]{inputenc}\n\\usepackage{geometry}\n\\geometry{a4paper, margin=1in}\n\n\\begin{document}\n\n\\section*{Schriftlicher Teil Bebauungsplan "Gewerbegebiet Himmelreich"\\\\\nStadt Laichingen, Gemarkung Feldstetten, Alb-Donau-Kreis, 19.06.2000}\n\n\\subsection*{1.2 Maß der baulichen Nutzung (§ 16 - 21 a BauNVO)}\n\n\\subsubsection*{1.2.1 Grundflächenzahl (§ 19 BauNVO)}\n- siehe Einträge im Lageplan -\n\n\\subsubsection*{1.2.2 Geschossflächenzahl (§ 20 BauNVO)}\n- siehe Einträge im Lageplan -\n\n\\subsubsection*{1.2.3 Höhe der baulichen Anlagen (§ 18 BauNVO)}\n- siehe Einträge im Lageplan -\nDie Gebäudehöhe wird gemessen von der Erdgeschossfußbodenhöhe (EFH) bis zur Schnittkante zwischen Außenwand und Dachhaut bzw. bis zur Oberkante Firstziegel.\n\n\\end{document}\n```\n\nDiese Festsetzungen umfasse

In [19]:
# T3.3) Textinformationen: Maß der baulichen Nutzung – WITH CONTEXT – IMG_PROMPTS
# Komplettes PDF
instruction = 'Extrahiere alle planungsrechtlichen Festsetzungen zum Thema "Maß der baulichen Nutzung". Liste am Ende die jeweiligen Seitenzahlen als Referenz auf. Output-Format: ###<Thema>: <Informationen> ### Referenzen: <Seitenzahlen>).'
async def run():
    return await ava.extractTextFromImagesWithContexts(instruction, [pdf_path], [context], img_type="pdf")
await runner.async_consistency_check(run)

['### Maß der baulichen Nutzung: 
1.2 Maß der baulichen Nutzung (§§ 16 - 21 BauNVO)
1.2.1 Grundflächenzahl (§ 19 BauNVO)
- siehe Einschriebe im Lageplan

1.2.2 Geschossflächenzahl (§ 20 BauNVO)
- siehe Einschriebe im Lageplan

1.2.3 Höhe der baulichen Anlagen (§ 18 BauNVO)
- siehe Einschriebe im Lageplan
- Die Gebäudehöhe wird gemessen von der Erdgeschossfußbodenhöhe (EFH) bis zur Schnittkante zwischen Außenwand und Dachhaut bzw. bis zur Oberkante Firstziegel.

### Referenzen: Seite 3']
#############################################
['### Maß der baulichen Nutzung:

1.2 Maß der baulichen Nutzung (§§ 16 - 21 BauNVO)
1.2.1 Grundflächenzahl (§ 19 BauNVO)
- siehe Einschreibung im Lageplan

1.2.2 Geschossflächenzahl (§ 20 BauNVO)
- siehe Einschreibung im Lageplan

1.2.3 Höhe der baulichen Anlagen (§ 18 BauNVO)
- siehe Einschreibung im Lageplan
- Die Gebäudehöhe wird gemessen von der Erdgeschossfußbodenhöhe (EFH) bis zur Schnittkante zwischen Außenwand und Dachhaut bzw. bis zur Oberkante Firs

[['### Maß der baulichen Nutzung: \n1.2 Maß der baulichen Nutzung (§§ 16 - 21 BauNVO)\n1.2.1 Grundflächenzahl (§ 19 BauNVO)\n- siehe Einschriebe im Lageplan\n\n1.2.2 Geschossflächenzahl (§ 20 BauNVO)\n- siehe Einschriebe im Lageplan\n\n1.2.3 Höhe der baulichen Anlagen (§ 18 BauNVO)\n- siehe Einschriebe im Lageplan\n- Die Gebäudehöhe wird gemessen von der Erdgeschossfußbodenhöhe (EFH) bis zur Schnittkante zwischen Außenwand und Dachhaut bzw. bis zur Oberkante Firstziegel.\n\n### Referenzen: Seite 3'],
 ['### Maß der baulichen Nutzung:\n\n1.2 Maß der baulichen Nutzung (§§ 16 - 21 BauNVO)\n1.2.1 Grundflächenzahl (§ 19 BauNVO)\n- siehe Einschreibung im Lageplan\n\n1.2.2 Geschossflächenzahl (§ 20 BauNVO)\n- siehe Einschreibung im Lageplan\n\n1.2.3 Höhe der baulichen Anlagen (§ 18 BauNVO)\n- siehe Einschreibung im Lageplan\n- Die Gebäudehöhe wird gemessen von der Erdgeschossfußbodenhöhe (EFH) bis zur Schnittkante zwischen Außenwand und Dachhaut bzw. bis zur Oberkante Firstziegel.\n\n### Refe

### T4) Bauweise, überbaubare Grundstücksflächen

1. text_prompts
2. img_prompts

In [25]:
%store -r msgbaunvo msgf11pdf
context = parser.text2prompts([msgbaunvo])

In [27]:
# T4.1) Textinformationen: Bauweise, überbaubare Grundstücksflächen – WITH CONTEXT – TEXT_PROMPTS
# Vector Store
instruction = 'Extrahiere alle planungsrechtlichen Festsetzungen zum Thema "Bauweise, überbaubare Grundstücksflächen".'

def run():
    return ava.similaritySearchWithContext(instruction, msgf11pdf, context)
runner.consistency_check(run)

['Hier sind die planungsrechtlichen Festsetzungen zum Thema "Bauweise, überbaubare Grundstücksflächen" aus dem Bebauungsplan "Gewerbegebiet Himmelreich":

### 1.3 Bauweise (§ 22 BauNVO)
- siehe Einschreibung im Lageplan -
- a = abweichende Bauweise. Es gilt die offene Bauweise, jedoch sind die Gebäudelängen nicht begrenzt.

### 1.4 Überbaubare Grundstücksfläche (§ 9 (1) 2 BauGB)
- Die überbaubare Grundstücksfläche wird durch die Festsetzung von Baugrenzen in der Planzeichnung bestimmt.
- Garagen im Sinne von § 12 BauNVO und Nebenanlagen im Sinne des § 14 BauNVO sowie Anlagen der Fremdwerbung und bauliche Anlagen, die nach Landesrecht in den Abstandsflächen zulässig sind oder zugelassen werden können, sind nur innerhalb der überbaubaren Grundstücksflächen zulässig.
- Der Geltungsbereich wird durch das Planzeichen im Lageplan begrenzt.

### 1.5 Von der Bebauung freizuhaltende Flächen und Sichtfelder (§ 9 (1) 10 BauGB)
- siehe Einschreibung im Lageplan -
- Auf den, zugunsten von EVS und d

[['Hier sind die planungsrechtlichen Festsetzungen zum Thema "Bauweise, überbaubare Grundstücksflächen" aus dem Bebauungsplan "Gewerbegebiet Himmelreich":\n\n### 1.3 Bauweise (§ 22 BauNVO)\n- siehe Einschreibung im Lageplan -\n- a = abweichende Bauweise. Es gilt die offene Bauweise, jedoch sind die Gebäudelängen nicht begrenzt.\n\n### 1.4 Überbaubare Grundstücksfläche (§ 9 (1) 2 BauGB)\n- Die überbaubare Grundstücksfläche wird durch die Festsetzung von Baugrenzen in der Planzeichnung bestimmt.\n- Garagen im Sinne von § 12 BauNVO und Nebenanlagen im Sinne des § 14 BauNVO sowie Anlagen der Fremdwerbung und bauliche Anlagen, die nach Landesrecht in den Abstandsflächen zulässig sind oder zugelassen werden können, sind nur innerhalb der überbaubaren Grundstücksflächen zulässig.\n- Der Geltungsbereich wird durch das Planzeichen im Lageplan begrenzt.\n\n### 1.5 Von der Bebauung freizuhaltende Flächen und Sichtfelder (§ 9 (1) 10 BauGB)\n- siehe Einschreibung im Lageplan -\n- Auf den, zugunsten

In [22]:
# T4.3) Textinformationen: Bauweise, überbaubare Grundstücksflächen – WITH CONTEXT – IMG_PROMPTS
# Komplettes PDF
instruction = 'Extrahiere alle planungsrechtlichen Festsetzungen zum Thema "Bauweise, überbaubare Grundstücksflächen". Liste am Ende die jeweiligen Seitenzahlen als Referenz auf. Output-Format: ###<Thema>: <Informationen> ### Referenzen: <Seitenzahlen>). '

async def run():
    return await ava.extractTextFromImagesWithContexts(instruction, [pdf_path], [context], img_type="pdf")
await runner.async_consistency_check(run)

['### Bauweise, überbaubare Grundstücksflächen:

1. **Bauweise (§ 22 BauNVO)**
   - Siehe Einschreibung im Lageplan.
   - Abweichende Bauweise. Es gilt die offene Bauweise, jedoch sind die Gebäudelängen nicht begrenzt.

2. **Überbaubare Grundstücksfläche (§ 9 (1) 2 BauGB)**
   - Die überbaubare Grundstücksfläche wird durch die Festsetzung von Baugrenzen in der Planzeichnung bestimmt.
   - Garagen im Sinne von § 12 BauNVO und Nebenanlagen im Sinne des § 14 BauNVO sowie Anlagen der Fremdwerbung und bauliche Anlagen, die nach Landesrecht in den Abstandsflächen zulässig sind oder zugelassen werden können, sind nur innerhalb der überbaubaren Grundstücksflächen zulässig.

### Referenzen:
- Seitenzahlen: 4']
#############################################
['### Bauweise, überbaubare Grundstücksflächen:

1.3 Bauweise (§ 22 BauNVO)
- Siehe Einträge im Lageplan.
- Abweichende Bauweise. Es gilt die offene Bauweise, jedoch sind die Gebäudelängen nicht begrenzt.

1.4 Überbaubare Grundstücksfläche (§ 

[['### Bauweise, überbaubare Grundstücksflächen:\n\n1. **Bauweise (§ 22 BauNVO)**\n   - Siehe Einschreibung im Lageplan.\n   - Abweichende Bauweise. Es gilt die offene Bauweise, jedoch sind die Gebäudelängen nicht begrenzt.\n\n2. **Überbaubare Grundstücksfläche (§ 9 (1) 2 BauGB)**\n   - Die überbaubare Grundstücksfläche wird durch die Festsetzung von Baugrenzen in der Planzeichnung bestimmt.\n   - Garagen im Sinne von § 12 BauNVO und Nebenanlagen im Sinne des § 14 BauNVO sowie Anlagen der Fremdwerbung und bauliche Anlagen, die nach Landesrecht in den Abstandsflächen zulässig sind oder zugelassen werden können, sind nur innerhalb der überbaubaren Grundstücksflächen zulässig.\n\n### Referenzen:\n- Seitenzahlen: 4'],
 ['### Bauweise, überbaubare Grundstücksflächen:\n\n1.3 Bauweise (§ 22 BauNVO)\n- Siehe Einträge im Lageplan.\n- Abweichende Bauweise. Es gilt die offene Bauweise, jedoch sind die Gebäudelängen nicht begrenzt.\n\n1.4 Überbaubare Grundstücksfläche (§ 9 (1) 2 BauGB)\n- Die über

### T5) Alle Themen in einem Prompt

* img_prompts

In [23]:
%store -r msgbaunvo msgf11pdf
context = parser.text2prompts([msgbaunvo])

In [28]:
# T5.3) Textinformationen: Alle Themen – WITH CONTEXT – IMG_PROMPTS
instruction = 'Extrahiere alle planungsrechtlichen Festsetzungen zu den folgenden Themen: "Art der baulichen Nutzung", "Maß der baulichen Nutzung", "Bauweise, überbaubare Grundstücksflächen". Liste am Ende die jeweiligen Seitenzahlen als Referenz auf. Output-Format: ###<Thema>: <Informationen> ### Referenzen: <Seitenzahlen>). '

async def run():
    return await ava.extractTextFromImagesWithContexts(instruction, [pdf_path], [context], img_type="pdf")
results = await runner.async_consistency_check(run)
msg232_f11_tt = results[0]
%store msg232_f11_tt

['### Art der baulichen Nutzung:
1.1 Eingeschränktes Gewerbegebiet (GEE) (§ 8 BauNVO i.V.m. § 1 (4) BauNVO)
   - Zulässig sind:
     - Gewerbebetriebe, die das Wohnen nicht wesentlich stören, wobei bei Einzelhandelsbetrieben (gem. § 1 (5) BauNVO i.V.m. § 1 (9) BauNVO) nur die nicht innenstadtrelevanten Sortimente Kfz, Motorräder, Mopeds, Fahrräder, Kfz-Zubehör, Rasenmäher, Landmaschinen, Fahrrad- und Motorradzubehör, Brennstoffe und Mineralölerzeugnisse zulässig sind.
     - Der Verkauf von auf dem Grundstück produzierten Waren auf einer untergeordneten Fläche ist zulässig.
   - Nicht zulässig sind:
     - Nutzungen gemäß § 8 (3) Nr. 3 BauNVO (Vergnügungsstätten) und gemäß § 1 (6) BauNVO nicht Bestandteil des Bebauungsplans und damit nicht zulässig.
     - Einzelhandelsbetriebe mit innenstadtrelevanten und bestimmten nicht innenstadtrelevanten Sortimenten sind gemäß § 1 (5) BauNVO i.V.m. § 1 (9) BauNVO nicht zulässig.

1.2 Gewerbegebiet (GE) (§ 8 BauNVO)
   - Zulässig sind:
     - Gewe

In [14]:
# T5.4.1) Textinformationen: Alle Themen – WITHOUT CONTEXT – IMG_PROMPTS
instruction = 'Extrahiere alle planungsrechtlichen Festsetzungen zu den folgenden Themen: "Art der baulichen Nutzung", "Maß der baulichen Nutzung", "Bauweise, überbaubare Grundstücksflächen". Liste am Ende die jeweiligen Seitenzahlen als Referenz auf. Output-Format: ###<Thema>: <Informationen> ### Referenzen: <Seitenzahlen>). '

async def run():
    return await ava.extractTextFromImages(instruction, [pdf_path], img_type="pdf")
await runner.async_consistency_check(run)

['### Art der baulichen Nutzung:
1.1 Eingeschränktes Gewerbegebiet (GEE) (§ 8 BauNVO i.V.m. § 1 (4) BauNVO)
1.1.1 Zulässig sind / zulässig ist:
1.1.1.1 Gewerbebetriebe, die das Wohnen nicht wesentlich stören, wobei bei Einzelhandelsbetrieben (gem. § 1 (5) BauNVO i.V.m. § 1 (9) BauNVO) nur die nicht innenstadtrelevanten Sortimente Kfz, Motorräder, Mopeds, Fahrräder, Kfz-Zubehör, Rasenmäher, Landmaschinen, Fahrrad- und Motorradzubehör, Brennstoffe und Mineralölerzeugnisse zulässig sind.
1.1.1.2 Der Verkauf von auf dem Grundstück produzierten Waren auf einer untergeordneten Fläche ist zulässig.
1.1.2 Nicht zulässig sind / nicht zulässig ist:
1.1.2.1 Nutzungen gemäß § 8 (3) Nr. 3 BauNVO (Vergnügungsstätten) sind gemäß § 1 (6) BauNVO nicht Bestandteil des Bebauungsplans und damit nicht zulässig.
1.1.2.2 Einzelhandelsbetriebe mit innenstadtrelevanten und bestimmten nicht innenstadtrelevanten Sortimenten sind gemäß § 1 (5) BauNVO i.V.m. § 1 (9) BauNVO nicht zulässig.
1.2 Gewerbegebiet (GE) (§

[['### Art der baulichen Nutzung:\n1.1 Eingeschränktes Gewerbegebiet (GEE) (§ 8 BauNVO i.V.m. § 1 (4) BauNVO)\n1.1.1 Zulässig sind / zulässig ist:\n1.1.1.1 Gewerbebetriebe, die das Wohnen nicht wesentlich stören, wobei bei Einzelhandelsbetrieben (gem. § 1 (5) BauNVO i.V.m. § 1 (9) BauNVO) nur die nicht innenstadtrelevanten Sortimente Kfz, Motorräder, Mopeds, Fahrräder, Kfz-Zubehör, Rasenmäher, Landmaschinen, Fahrrad- und Motorradzubehör, Brennstoffe und Mineralölerzeugnisse zulässig sind.\n1.1.1.2 Der Verkauf von auf dem Grundstück produzierten Waren auf einer untergeordneten Fläche ist zulässig.\n1.1.2 Nicht zulässig sind / nicht zulässig ist:\n1.1.2.1 Nutzungen gemäß § 8 (3) Nr. 3 BauNVO (Vergnügungsstätten) sind gemäß § 1 (6) BauNVO nicht Bestandteil des Bebauungsplans und damit nicht zulässig.\n1.1.2.2 Einzelhandelsbetriebe mit innenstadtrelevanten und bestimmten nicht innenstadtrelevanten Sortimenten sind gemäß § 1 (5) BauNVO i.V.m. § 1 (9) BauNVO nicht zulässig.\n1.2 Gewerbegebie